In [2]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv

from trd_agent.tools.search_perplexity import PerplexitySearchTool
from trd_agent.tools.search_tavily import TavilySearchTool

import asyncio
import os
from typing import Dict, List
from IPython.display import display, Markdown

In [3]:
NUMBER_OF_SEARCH = 3


In [14]:
import json
search_tool_perplexity = PerplexitySearchTool()
search_tool_tavily = TavilySearchTool()
@function_tool
def perplexity_search(query: str) -> str:
    """
    Perform Perplexity web search and return JSON results.

    Args:
        query: Natural-language search query.
    """
    result = search_tool_perplexity.run(query=query)
    # Better to return JSON string than raw dict
    return json.dumps(result)


search_instruction = """You are a research assistant. Given a search term, search the web for that term, 
and produce a concise summary of the results. 
The summary will be sonsumed by someone synthesizing a full report, so it is vital that: 
+ The summary must be 2-3 paragraphs and less than 300 words, writen succintly, no complete sentence or good grammar. 
+ The summary captures the main points, ignore any fluff, no additional commentary."""


search_agent = Agent(
    name="Search agent",
    instructions=search_instruction,
    tools = [perplexity_search],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required")
)

In [7]:
search_plan_instruction = f"""You are deep research assistant.
Given a query, come up with a set of web searches to best answer the query. 
Output {NUMBER_OF_SEARCH} terms to query for."""

#use BaseModel like a schema, to describe a structure, which later ask model to return
class WebSearchItem(BaseModel):
    reason: str
    "your reasoning for why this search is important to the query"#ensure these information is provided to the model

    query: str
    "the search term to use for the web search"

class WebSearchPlan(BaseModel):
    searches: List[WebSearchItem]
    """A list of web searches to perform to best answer the query"""

planner_agent = Agent(
    name="PlannerAgent",
    instructions=search_plan_instruction,
    model="gpt-4o-mini",
    output_type=WebSearchPlan

)

In [6]:
message = "List AI Agent frameworks in 2025"
with trace("PlanAndSearch"):
    result = await Runner.run(planner_agent, message)
    print (result.final_output)

searches=[WebSearchItem(reason='To find the latest AI agent frameworks developed or popularized by 2025.', query='AI agent frameworks 2025'), WebSearchItem(reason='To gather insights on the technologies and methodologies used in AI agents as of 2025.', query='top AI technologies 2025'), WebSearchItem(reason='To discover industry reports and surveys that list and analyze AI agent frameworks available in 2025.', query='AI agent frameworks industry report 2025')]


## declare executor - perform search

In [8]:
search_exec_instruction = """You are researcher taksed with writing cohesive report for a research query. 
You are given the original query and some initial research done by a research assistant.
Come up with an outline for the report describing the structure and flow of the report.
Then generate the report and return that as your final output.
The final output should be in markdown format, lengthly, and in detailed. Aim for 10 pages of content with at least 1000 words.
"""

In [10]:
class ReportData(BaseModel):
    short_summary: str
    """A short 2~3 sentence summary of the findings"""

    markdown_report: str
    """The final report"""

    follow_up_question: list[str]
    """Suggested topics to research further"""

writer_agent = Agent(
    name="WriterAgent",
    instructions=search_exec_instruction,
    model="gpt-4o-mini",
    output_type=ReportData
)


#  Email agent, copied from previous lagb

In [23]:
subject_instruction = "Given a message, you write a subject for a cold sales email that is likely to get a response"

html_instruction = "Given a text email body which may have some markdown, convert it to an HTML email with simple, clear, compeling layout and design"


subject_writer = Agent(name="Email subject writer", instructions=subject_instruction, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name = "subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instruction, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to an HTML email body")

@function_tool
def send_html_email(subject: str, html_body: str)->Dict[str,str]:
    """ Send out an email with given subject and HTML body to all sales prospect """
    print ("pretending to send an html email")
    print ("sending....")
    print (f"\n\n#############{subject}###########\n\n")
    print (f"\n\n#############{html_body}###########\n\n")
    print ("done!")
    return {"status": "success"}

email_tools = [subject_tool, html_tool, send_html_email]

email_tools

email_instruction="""You are email formatter and sender.
Given body of an email to be sent, you first use the subject_writer tool to write subject for the email.
Then, use the html_converter tool to convert to body to HTML.
Finally, use send_html_email tool to send out the email with subject and HTML body."""
emailer_agent = Agent(name="Email Manager", instructions=email_instruction, tools=email_tools, model="gpt-4o-mini", handoff_description="Convert an email to HTML and send it")

handoff = [emailer_agent]

# Use planner_agent and search_agent

In [24]:
async def plan_searches(query:str):
    """Use planner_agent to plot out searches terms"""
    print ("Planning se3arches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print (f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """Call search() for each search item in search plan"""
    num_completed = 0
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)

    print (f"Finish searching")
    return results

async def search(item: WebSearchItem):
    """ Use search agent to run a web search for each item in search plan"""
    input = f"Search term: {item.query} \n Search strategy reasoning: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [28]:
async def write_report(query: str, search_results: list[str]):
    """Use writer agent to write a report based on the search results"""
    print ("Thinking about the report..")
    input = f"Original query: {query}\n summarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print ("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """use email agent to send email"""
    print("writing email...")
    result = await Runner.run(emailer_agent, report.markdown_report)
    print ("Email sent")
    return report

# Show time

In [29]:
query = "Latest AI Agent frameworks in 2025"

In [30]:
with trace("Research trace planned-write-email"):
    print ("Starting search")
    search_plan = await plan_searches(query)
    search_result = await perform_searches(search_plan)
    report = await write_report(query, search_result)
    await send_email(report)
    print ("Done!")

Starting search
Planning se3arches...
Will perform 3 searches
Finish searching
Thinking about the report..
Finished writing report
writing email...
pretending to send an html email
sending....


#############Unlock Success: Discover the Top AI Agent Frameworks of 2025###########




#############<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Report on Latest AI Agent Frameworks in 2025</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            line-height: 1.6;
            padding: 20px;
            background-color: #f4f4f4;
        }
        .container {
            max-width: 800px;
            margin: auto;
            background: #fff;
            padding: 20px;
            border-radius: 5px;
            box-shadow: 0 0 10px rgba(0,0,0,.1);
        }
        h1, h2, h3, h4 {
            color: #333;
        }
        h1 {
            text-align